In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from numpy import average
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import statistics


### Most updated USWTDB version 3

In [2]:
wind = gpd.read_file('../data/USWTDB/uswtdb_v3_2_20201014.geojson')

In [3]:
wind.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 65548 entries, 0 to 65547
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   case_id     65548 non-null  float64 
 1   t_state     65548 non-null  object  
 2   p_name      65548 non-null  object  
 3   p_year      64055 non-null  float64 
 4   p_tnum      65548 non-null  float64 
 5   p_cap       60580 non-null  float64 
 6   t_manu      59569 non-null  object  
 7   t_model     59245 non-null  object  
 8   t_cap       59571 non-null  float64 
 9   t_hh        58629 non-null  float64 
 10  t_rd        58911 non-null  float64 
 11  t_rsa       58911 non-null  float64 
 12  t_ttlh      58629 non-null  float64 
 13  t_conf_atr  65548 non-null  float64 
 14  t_conf_loc  65548 non-null  float64 
 15  xlong       65548 non-null  float64 
 16  ylat        65548 non-null  float64 
 17  geometry    65548 non-null  geometry
dtypes: float64(13), geometry(1), object(4)

In [4]:
manufacturers = wind.t_manu.value_counts().to_frame().reset_index()
manufacturers['index'].iloc[40:68]
manufacturers.head(20)

,index,t_manu
0,GE Wind,25858
1,Vestas,13904
2,Siemens,5144
3,Gamesa,3076
4,Mitsubishi,2791
5,Suzlon,1316
6,Siemens Gamesa Renewable Energy,1212
7,Nordex,1145
8,Acciona,758
9,Clipper,681


In [5]:
wind.shape

(65548, 18)

In [6]:
year_install = wind.p_year.value_counts().to_frame().reset_index()

In [7]:
year_install =  year_install.sort_values(by = 'index')

In [8]:
year_install #wind turbines by installation year still operating

,index,p_year
36,1981.0,11
17,1982.0,937
22,1983.0,433
30,1984.0,155
19,1985.0,714
31,1986.0,151
28,1987.0,213
26,1988.0,277
25,1989.0,288
24,1990.0,327


### US Wind Turbine Data from 2014

In [9]:
wind_old = pd.read_csv('../data/Onshore_Industrial_Wind_Turbine_Locations_for_the_United_States_to_March_2014.csv')

In [10]:
wind_old.columns

Index(['FID', 'unique_id', 'site_name', 'total_turb', 'on_year', 'year_range',
       'on_year_s', 'manufac', 'model', 'type_tower', 'decommiss',
       'MW_turbine', 'total_cpcy', 'total_ht', 'tower_h', 'blade_l',
       'rotor_dia', 'rotor_s_a', 'lat_DD', 'long_DD', 'state', 'county',
       'conf_attr', 'conf_loc', 'WENDI_name', 'EIA_name', 'FAA_jdate',
       'FAA_AGL', 'FAA_ORS', 'image_name', 'image_year', 'comments'],
      dtype='object')

In [11]:
wind_old['geometry'] = wind_old.apply(lambda x: Point((float(x.long_DD), 
                                                         float(x.lat_DD))), 
                                        axis=1)

In [12]:
wind_old.head(1)

,FID,unique_id,site_name,total_turb,on_year,year_range,on_year_s,manufac,model,type_tower,...,conf_loc,WENDI_name,EIA_name,FAA_jdate,FAA_AGL,FAA_ORS,image_name,image_year,comments,geometry
0,0,982,unknown Gilliam County,2,unknown,unknown,-99999,unknown,unknown,unknown,...,0,unknown,unknown,2013192,124.09,41-020577,Bing Maps Aerial,unknown,"FAA lists as Dismantle, google temporal analys...",POINT (-120.298786 45.653994)


In [13]:
wind_old_geo = gpd.GeoDataFrame(wind_old, 
                           crs = wind.crs, 
                           geometry = wind_old['geometry'])

In [14]:
wind_old_geo.columns

Index(['FID', 'unique_id', 'site_name', 'total_turb', 'on_year', 'year_range',
       'on_year_s', 'manufac', 'model', 'type_tower', 'decommiss',
       'MW_turbine', 'total_cpcy', 'total_ht', 'tower_h', 'blade_l',
       'rotor_dia', 'rotor_s_a', 'lat_DD', 'long_DD', 'state', 'county',
       'conf_attr', 'conf_loc', 'WENDI_name', 'EIA_name', 'FAA_jdate',
       'FAA_AGL', 'FAA_ORS', 'image_name', 'image_year', 'comments',
       'geometry'],
      dtype='object')

In [15]:
wind_old_geo = wind_old_geo.replace(-99999, np.nan)

In [16]:
wind_2014_by_year = wind_old_geo.groupby(['on_year_s']).unique_id.count().to_frame().reset_index()
wind_2014_by_year

,on_year_s,unique_id
0,1981.0,3679
1,1982.0,3635
2,1983.0,1343
3,1984.0,668
4,1985.0,602
5,1986.0,452
6,1987.0,264
7,1988.0,302
8,1989.0,440
9,1990.0,59


In [17]:
wind_old_geo['tower_h_ft'] = wind_old_geo['tower_h']*3.2808

In [18]:
#create bins/year group column
bins= [1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015]
labels = ['1980-85','1985-90','1990-95','1995-00','2000-05','2005-10','2010-15']
wind_old_geo['year_group'] = pd.cut(wind_old_geo['on_year_s'], bins=bins, labels=labels, right=False)
wind_old_geo.head()

,FID,unique_id,site_name,total_turb,on_year,year_range,on_year_s,manufac,model,type_tower,...,EIA_name,FAA_jdate,FAA_AGL,FAA_ORS,image_name,image_year,comments,geometry,tower_h_ft,year_group
0,0,982,unknown Gilliam County,2,unknown,unknown,NaN,unknown,unknown,unknown,...,unknown,2013192.0,124.09,41-020577,Bing Maps Aerial,unknown,"FAA lists as Dismantle, google temporal analys...",POINT (-120.29879 45.65399),NaN,NaN
1,1,1065,unknown Gilliam County,2,unknown,unknown,NaN,unknown,unknown,unknown,...,unknown,2013192.0,124.09,41-020578,Bing Maps Aerial,unknown,"FAA lists as Dismantle, google temporal analys...",POINT (-120.30599 45.66349),NaN,NaN
2,2,1895,Banner Wind Project,2,2008,no,2008.0,Entegrity,EW50,trestle,...,unknown,2009065.0,38.72,02-020064,Bing Maps Aerial,unknown,"trestle turbines removed, new monopoles instal...",POINT (-165.42748 64.56990),102.03288,2005-10
3,3,1897,Banner Wind Project,2,2008,no,2008.0,Entegrity,EW50,trestle,...,unknown,2009065.0,38.72,02-020070,Bing Maps Aerial,unknown,"trestle turbines removed, new monopoles instal...",POINT (-165.42638 64.57090),102.03288,2005-10
4,4,2608,unknown Tehachapi Wind Resource Area 1,549,1982_1990,yes,1982.0,unknown,unknown,monopole,...,unknown,NaN,NaN,unknown,NAIP,2012,,POINT (-118.32149 35.05990),NaN,1980-85


In [19]:
wind_2014_tower_height = wind_old_geo.groupby('year_group').tower_h_ft.describe()
#wind_2014_tower_height.to_csv('wind_2014_tower_height.csv')
wind_2014_tower_height.transpose().to_csv('wind_2014_tower_height2.csv')
wth2 = wind_2014_tower_height.transpose().stack().reset_index()
wth2.columns = [['type', 'year_group', 'calc']]
#wth2.to_csv('wind_2014_tower_height2.csv')
wind_2014_tower_height

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,1204.0,78.142716,18.609394,59.05440,59.71056,74.80224,78.73920,119.7492
1985-90,1195.0,72.057075,9.559587,59.71056,59.71056,78.73920,80.70768,82.0200
1990-95,416.0,95.302508,7.347925,80.05152,98.42400,98.42400,98.42400,141.0744
1995-00,1196.0,184.401808,23.849565,78.08304,164.04000,180.44400,206.69040,255.9024
2000-05,3801.0,205.641165,28.969916,78.08304,196.84800,212.26776,219.81360,278.8680
2005-10,17045.0,255.697814,18.853714,98.42400,262.46400,262.46400,262.46400,344.4840
2010-15,13492.0,272.567463,26.055588,63.97560,262.46400,262.46400,262.46400,393.6960


In [20]:
wind_old_geo.groupby('year_group').tower_h_ft.max()

year_group
1980-85    119.7492
1985-90     82.0200
1990-95    141.0744
1995-00    255.9024
2000-05    278.8680
2005-10    344.4840
2010-15    393.6960
Name: tower_h_ft, dtype: float64

In [21]:
one = wind_old_geo[wind_old_geo['year_group'] == '1980-85']
two = wind_old_geo[wind_old_geo['year_group'] == '1985-90']

In [22]:
print(statistics.mode(list(one['tower_h'])))
print(statistics.mode(list(two['tower_h'])))

22.8
18.2


In [23]:
one = one.tower_h.value_counts().to_frame().reset_index()
one.columns = ['tower_height', 'count']

In [24]:
one

,tower_height,count
0,22.8,453
1,18.2,286
2,24.0,252
3,36.5,175
4,18.0,26
5,20.0,12


In [25]:
two.tower_h.value_counts()

18.2    440
24.0    304
24.6    210
23.0    116
25.0     96
24.5     17
24.3     12
Name: tower_h, dtype: int64

In [26]:
distribution = one['tower_height']
weights = one['count']

In [27]:
weight_avg = round(average(one['tower_height'], weights = one['count']), 2)

In [28]:
weight_avg

23.82

In [29]:
wind_2014_rate_cap = wind_old_geo.groupby('year_group').MW_turbine.describe()
wind_2014_rate_cap.to_csv('wind_2014_rate_cap.csv')

In [30]:
wind.groupby('p_year').t_hh.mean()

p_year
1981.0          NaN
1982.0          NaN
1983.0    22.800000
1984.0    24.000000
1985.0    24.386341
1986.0    25.285714
1987.0          NaN
1988.0    23.000000
1989.0          NaN
1990.0    30.000000
1991.0          NaN
1992.0    41.500000
1994.0    40.000000
1995.0    39.651163
1996.0    50.000000
1997.0    41.647059
1998.0    52.080645
1999.0    57.633149
2000.0    57.180556
2001.0    58.414576
2002.0    62.699782
2003.0    69.234783
2004.0    65.421779
2005.0    75.148028
2006.0    75.845436
2007.0    78.132560
2008.0    78.195780
2009.0    78.621082
2010.0    79.325188
2011.0    80.877153
2012.0    83.711371
2013.0    80.418803
2014.0    82.666133
2015.0    82.456762
2016.0    82.940776
2017.0    86.010600
2018.0    88.251646
2019.0    90.078615
2020.0    90.112127
Name: t_hh, dtype: float64

In [31]:
hub_height_year = wind.groupby(['p_year', 't_manu']).t_hh.mean().to_frame().reset_index()
hub_height_year.head()

,p_year,t_manu,t_hh
0,1982.0,Lolland,NaN
1,1982.0,Micon,NaN
2,1982.0,Vestas,NaN
3,1983.0,Vestas,22.8
4,1984.0,Bonus,24.0


In [32]:
statistics.mode(list(wind['t_hh']))

80.0

In [33]:
turbine_count_year = wind.groupby(['p_year', 't_manu']).case_id.count().to_frame().reset_index()

In [34]:
turbine_count_year = turbine_count_year.sort_values(by=['t_manu', 'p_year'])

In [35]:
turbine_count_year

,p_year,t_manu,case_id
127,2009.0,AAER,2
152,2010.0,AAER,1
29,1997.0,AOC,1
39,1999.0,AOC,5
46,2000.0,AOC,2
...,...,...,...
27,1995.0,Windmatic,1
232,2012.0,Windmatic,5
34,1997.0,Zond,12
38,1998.0,Zond,3


In [36]:
turbine_count_year.to_csv('turbine_year.csv')

In [37]:
wind.groupby(['p_year', 't_state']).case_id.count()

p_year  t_state
1981.0  CA          11
1982.0  CA         937
1983.0  CA         432
        OK           1
1984.0  CA         155
                  ... 
2020.0  TX         882
        VA           2
        WA          38
        WV          20
        WY         153
Name: case_id, Length: 502, dtype: int64

In [38]:
wind[wind['p_year'] == 1981.0].shape

(11, 18)

In [39]:
wind_v1 = pd.read_csv('../data/uswtdb_v1_0_20180419.csv')

In [40]:
wind_v1[wind_v1['p_year'] == 1981].shape

(537, 24)

In [41]:
wind_v1_year = wind_v1.p_year.value_counts().to_frame().reset_index().sort_values('index') #released in 2018
wind_v1_year.columns = ['year', 'count_v1']
year_install.columns = ['year', 'count_v3']

In [42]:
v1_v3 = pd.merge(wind_v1_year, year_install, on = 'year', how = 'right')
v1_v3

,year,count_v1,count_v3
0,1981,537.0,11
1,1982,1910.0,937
2,1983,979.0,433
3,1984,216.0,155
4,1985,1467.0,714
5,1986,311.0,151
6,1987,406.0,213
7,1988,282.0,277
8,1989,NaN,288
9,1990,432.0,327


In [43]:
#create bins/year group column
bins= [1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2021]
labels = ['1980-85','1985-90','1990-95','1995-00','2000-05','2005-10','2010-15', '2015-20']
wind['year_group'] = pd.cut(wind['p_year'], bins=bins, labels=labels, right=False)
wind.head()

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90


In [44]:
wind.groupby('year_group').size()

year_group
1980-85     1536
1985-90     1643
1990-95      360
1995-00     1381
2000-05     3888
2005-10    17112
2010-15    16342
2015-20    21793
dtype: int64

In [45]:
wind_2000 = wind[wind['year_group'].isin(['2000-05', '2005-10', '2010-15', '2015-20'])]
wind_2000.year_group.value_counts()

2015-20    21793
2005-10    17112
2010-15    16342
2000-05     3888
1995-00        0
1990-95        0
1985-90        0
1980-85        0
Name: year_group, dtype: int64

In [46]:
wind_2000.groupby('year_group').t_hh.describe()

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-90,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-95,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,3879.0,62.694380,9.358704,24.0,55.0,65.0,69.0,80.0
2005-10,17028.0,77.833081,5.849805,22.8,80.0,80.0,80.0,105.0
2010-15,16238.0,82.033483,7.414646,23.0,80.0,80.0,80.0,103.0
2015-20,19144.0,86.042682,7.274685,30.0,80.0,85.0,90.0,131.0


In [47]:
wind_2000.groupby(['year_group', 't_state']).size().to_frame().reset_index().tail(50)

,year_group,t_state,0
318,2010-15,WA,556
319,2010-15,WI,113
320,2010-15,WV,151
321,2010-15,WY,185
322,2015-20,AK,9
323,2015-20,AR,0
324,2015-20,AZ,17
325,2015-20,CA,276
326,2015-20,CO,1029
327,2015-20,CT,2


In [48]:
wind[wind['t_manu'] == 'Vestas']

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59728,3088836.0,OK,Wildhorse Mountain Wind Facility,2019.0,29.0,100.05,Vestas,V136-3.45,3450.0,105.0,136.0,14526.72,173.1,3.0,1.0,-95.242958,34.588802,POINT (-95.24296 34.58880),2015-20
60585,3090405.0,TX,Woodward Mountain II,2001.0,115.0,75.90,Vestas,V47-0.66,660.0,65.0,47.0,1734.94,88.4,3.0,3.0,-102.366089,30.966394,POINT (-102.36609 30.96639),2000-05
60628,3090403.0,TX,Woodward Mountain II,2001.0,115.0,75.90,Vestas,V47-0.66,660.0,65.0,47.0,1734.94,88.4,3.0,3.0,-102.364494,30.967396,POINT (-102.36449 30.96740),2000-05
60646,3090404.0,TX,Woodward Mountain II,2001.0,115.0,75.90,Vestas,V47-0.66,660.0,65.0,47.0,1734.94,88.4,3.0,3.0,-102.365288,30.966894,POINT (-102.36529 30.96689),2000-05


In [49]:
wind.columns

Index(['case_id', 't_state', 'p_name', 'p_year', 'p_tnum', 'p_cap', 't_manu',
       't_model', 't_cap', 't_hh', 't_rd', 't_rsa', 't_ttlh', 't_conf_atr',
       't_conf_loc', 'xlong', 'ylat', 'geometry', 'year_group'],
      dtype='object')

In [50]:
wind['rated_cap_mw'] = wind['t_cap']/1000

In [51]:
wind['t_hh_ft'] = wind['t_hh']*3.2808

In [52]:
wind.head(30)

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,...,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group,rated_cap_mw,t_hh_ft
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90,0.095,NaN
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90,0.095,NaN
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90,0.095,NaN
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90,0.095,NaN
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90,0.095,NaN
5,3001247.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368187,35.075794,POINT (-118.36819 35.07579),1985-90,0.095,NaN
6,3009410.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368088,35.075893,POINT (-118.36809 35.07589),1985-90,0.095,NaN
7,3073377.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.352165,35.088856,POINT (-118.35216 35.08886),1985-90,0.095,NaN
8,3072663.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368202,35.075695,POINT (-118.36820 35.07570),1985-90,0.095,NaN
9,3072670.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368408,35.075542,POINT (-118.36841 35.07554),1985-90,0.095,NaN


In [53]:
onshore_wind = wind[wind['rated_cap_mw'] < 6.0]
onshore_wind.rated_cap_mw.describe()

count    59564.000000
mean         1.828285
std          0.667002
min          0.050000
25%          1.500000
50%          1.800000
75%          2.300000
max          4.200000
Name: rated_cap_mw, dtype: float64

In [54]:
wind_new_tower_height = onshore_wind.groupby('year_group').t_hh_ft.describe()

In [55]:
wind_2014_tower_height = wind_2014_tower_height.reset_index()

In [56]:
last = wind_new_tower_height.iloc[7:]
last = last.reset_index()

In [57]:
tower_height_combo = wind_2014_tower_height.append(last)

In [58]:
tower_height_combo.to_csv('tower_height_combo.csv')

In [59]:
tower_height_combo

,year_group,count,mean,std,min,25%,50%,75%,max
0,1980-85,1204.0,78.142716,18.609394,59.05440,59.71056,74.80224,78.73920,119.7492
1,1985-90,1195.0,72.057075,9.559587,59.71056,59.71056,78.73920,80.70768,82.0200
2,1990-95,416.0,95.302508,7.347925,80.05152,98.42400,98.42400,98.42400,141.0744
3,1995-00,1196.0,184.401808,23.849565,78.08304,164.04000,180.44400,206.69040,255.9024
4,2000-05,3801.0,205.641165,28.969916,78.08304,196.84800,212.26776,219.81360,278.8680
5,2005-10,17045.0,255.697814,18.853714,98.42400,262.46400,262.46400,262.46400,344.4840
6,2010-15,13492.0,272.567463,26.055588,63.97560,262.46400,262.46400,262.46400,393.6960
0,2015-20,19137.0,282.263166,23.832966,98.42400,262.46400,278.86800,295.27200,429.7848


In [60]:
wind[wind['t_hh'] == 131.0]

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,...,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group,rated_cap_mw,t_hh_ft
3542,3088804.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,1.0,-77.702141,41.906441,POINT (-77.70214 41.90644),2015-20,3.6,429.7848
3545,3089762.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,3.0,-77.686577,41.885166,POINT (-77.68658 41.88517),2015-20,3.6,429.7848
3547,3088659.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,3.0,-77.690758,41.903679,POINT (-77.69076 41.90368),2015-20,3.6,429.7848
3548,3089358.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,3.0,-77.691467,41.900085,POINT (-77.69147 41.90008),2015-20,3.6,429.7848
3549,3089490.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,3.0,-77.664841,41.900620,POINT (-77.66484 41.90062),2015-20,3.6,429.7848
3550,3089073.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,3.0,-77.677582,41.902752,POINT (-77.67758 41.90275),2015-20,3.6,429.7848
3551,3089301.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,3.0,-77.680588,41.864742,POINT (-77.68059 41.86474),2015-20,3.6,429.7848
3552,3089788.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,1.0,-77.662781,41.890270,POINT (-77.66278 41.89027),2015-20,3.6,429.7848
3553,3089541.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,1.0,-77.687531,41.892868,POINT (-77.68753 41.89287),2015-20,3.6,429.7848
3555,3089624.0,PA,Big Level,2019.0,25.0,90.0,GE Wind,GE3.6-137,3600.0,131.0,...,14741.14,199.6,3.0,1.0,-77.693039,41.907566,POINT (-77.69304 41.90757),2015-20,3.6,429.7848


In [61]:
wind_new_rated_cap = onshore_wind.groupby('year_group').rated_cap_mw.describe()
wind_new_rated_cap

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,591.0,0.065098,0.001872,0.060,0.065,0.065,0.065,0.108
1985-90,1075.0,0.106885,0.061423,0.065,0.090,0.105,0.108,1.000
1990-95,360.0,0.248264,0.074933,0.225,0.225,0.225,0.225,0.500
1995-00,1178.0,0.700836,0.146206,0.050,0.700,0.750,0.750,2.000
2000-05,3886.0,1.062082,0.411278,0.050,0.660,1.000,1.500,3.000
2005-10,17049.0,1.658130,0.416915,0.050,1.500,1.500,1.800,3.000
2010-15,16273.0,1.910421,0.441731,0.050,1.600,1.800,2.300,3.600
2015-20,19145.0,2.316092,0.500577,0.099,2.000,2.200,2.500,4.200


In [62]:
wind_2014_rate_cap = wind_2014_rate_cap.iloc[0:6].reset_index()

In [63]:
wind_new_rated_cap = wind_new_rated_cap.iloc[6:].reset_index()

In [64]:
rated_cap_combo = wind_2014_rate_cap.append(wind_new_rated_cap)

In [65]:
rated_cap_combo.to_csv('rated_cap_combo.csv')

In [66]:
rated_cap_combo

,year_group,count,mean,std,min,25%,50%,75%,max
0,1980-85,4497.0,0.096634,0.033205,0.040,0.100,0.100,0.100,0.40
1,1985-90,1640.0,0.112293,0.065962,0.065,0.065,0.100,0.105,0.33
2,1990-95,416.0,0.248630,0.051484,0.200,0.225,0.225,0.225,0.36
3,1995-00,1310.0,0.674065,0.176632,0.050,0.660,0.750,0.750,1.75
4,2000-05,3882.0,1.078297,0.407601,0.065,0.660,1.000,1.500,3.00
5,2005-10,17109.0,1.667116,0.420055,0.050,1.500,1.500,2.000,3.00
0,2010-15,16273.0,1.910421,0.441731,0.050,1.600,1.800,2.300,3.60
1,2015-20,19145.0,2.316092,0.500577,0.099,2.000,2.200,2.500,4.20


In [67]:
wind[wind['p_year'] == 2019.0]

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,...,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group,rated_cap_mw,t_hh_ft
1398,3089642.0,NH,Antrim,2019.0,9.0,28.8,Siemens,SWT-3.2-113,3200.0,92.5,...,10028.75,149.0,3.0,3.0,-72.007820,43.067608,POINT (-72.00782 43.06761),2015-20,3.2,303.4740
1399,3089313.0,NH,Antrim,2019.0,9.0,28.8,Siemens,SWT-3.2-113,3200.0,92.5,...,10028.75,149.0,3.0,3.0,-72.006195,43.064255,POINT (-72.00620 43.06426),2015-20,3.2,303.4740
1400,3089591.0,NH,Antrim,2019.0,9.0,28.8,Siemens,SWT-3.2-113,3200.0,92.5,...,10028.75,149.0,3.0,3.0,-72.019501,43.056614,POINT (-72.01950 43.05661),2015-20,3.2,303.4740
1401,3088892.0,NH,Antrim,2019.0,9.0,28.8,Siemens Gamesa Renewable Energy,SWT-3.2-113,3200.0,79.5,...,10028.75,135.9,3.0,3.0,-72.009064,43.061455,POINT (-72.00906 43.06146),2015-20,3.2,260.8236
1402,3089227.0,NH,Antrim,2019.0,9.0,28.8,Siemens,SWT-3.2-113,3200.0,92.5,...,10028.75,149.0,3.0,3.0,-72.023994,43.043133,POINT (-72.02399 43.04313),2015-20,3.2,303.4740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64310,3102321.0,CA,unknown Solano County,2019.0,2.0,NaN,None,None,NaN,NaN,...,NaN,NaN,1.0,3.0,-121.781288,38.488888,POINT (-121.78129 38.48889),2015-20,NaN,NaN
65152,3091670.0,TX,unknown Upton County,2019.0,1.0,NaN,None,None,NaN,NaN,...,NaN,NaN,1.0,3.0,-102.157959,31.279547,POINT (-102.15796 31.27955),2015-20,NaN,NaN
65192,3090176.0,MI,unknown Wayne County,2019.0,3.0,NaN,None,None,NaN,NaN,...,NaN,NaN,1.0,3.0,-83.161636,42.191433,POINT (-83.16164 42.19143),2015-20,NaN,NaN
65193,3090056.0,MI,unknown Wayne County,2019.0,3.0,NaN,None,None,NaN,NaN,...,NaN,NaN,1.0,3.0,-83.161552,42.192009,POINT (-83.16155 42.19201),2015-20,NaN,NaN


In [68]:
top_manu_turbine_count = onshore_wind.t_manu.value_counts().to_frame().reset_index()

In [90]:
top_manu_turbine_count.head(60)

,index,t_manu
0,GE Wind,25853
1,Vestas,13799
2,Siemens,5144
3,Gamesa,3076
4,Mitsubishi,2791
5,Suzlon,1316
6,Nordex,1145
7,Siemens Gamesa Renewable Energy,1135
8,Acciona,758
9,Clipper,681


In [70]:
top_manu_rated_cap = onshore_wind.groupby('t_manu').rated_cap_mw.sum().to_frame().reset_index()

In [71]:
top_manu_rated_cap = top_manu_rated_cap.sort_values(by='rated_cap_mw', ascending = False)

In [72]:
top_manu_rated_cap.head(20)

,t_manu,rated_cap_mw
21,GE Wind,46130.430
60,Vestas,26073.410
49,Siemens,12039.014
22,Gamesa,5707.570
32,Mitsubishi,3513.800
35,Nordex,3344.410
50,Siemens Gamesa Renewable Energy,2827.395
54,Suzlon,2644.050
11,Clipper,1702.500
3,Acciona,1372.500


In [73]:
wind.head(20)

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,...,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group,rated_cap_mw,t_hh_ft
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90,0.095,NaN
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90,0.095,NaN
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90,0.095,NaN
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90,0.095,NaN
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90,0.095,NaN
5,3001247.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368187,35.075794,POINT (-118.36819 35.07579),1985-90,0.095,NaN
6,3009410.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368088,35.075893,POINT (-118.36809 35.07589),1985-90,0.095,NaN
7,3073377.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.352165,35.088856,POINT (-118.35216 35.08886),1985-90,0.095,NaN
8,3072663.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368202,35.075695,POINT (-118.36820 35.07570),1985-90,0.095,NaN
9,3072670.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,...,NaN,NaN,2.0,3.0,-118.368408,35.075542,POINT (-118.36841 35.07554),1985-90,0.095,NaN


In [74]:
wind.groupby('year_group').t_rd.describe()

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,586.0,15.574403,0.504401,15.3,15.3,15.3,15.3,16.5
1985-90,509.0,18.106090,3.533604,11.0,16.0,17.0,19.4,47.0
1990-95,339.0,27.442478,2.390581,27.0,27.0,27.0,27.0,42.0
1995-00,1170.0,47.157521,5.913125,13.4,47.0,48.0,50.0,66.0
2000-05,3879.0,60.210621,14.071819,15.0,47.0,61.4,70.5,103.0
2005-10,17035.0,80.137323,10.134555,14.0,77.0,77.0,87.0,108.0
2010-15,16242.0,91.693732,10.948582,17.0,82.5,90.0,100.0,120.0
2015-20,19144.0,112.685954,11.051270,20.8,107.0,110.0,117.0,154.0


In [75]:
wind.groupby('year_group').t_rsa.describe()

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,586.0,190.703208,12.597420,183.85,183.85,183.85,183.85,213.82
1985-90,509.0,267.264676,129.012153,95.03,201.06,226.98,295.59,1734.94
1990-95,339.0,593.703009,114.049734,572.56,572.56,572.56,572.56,1385.44
1995-00,1170.0,1774.033632,346.323157,141.03,1734.94,1809.56,1963.50,3421.19
2000-05,3879.0,3002.217321,1411.479104,176.71,1734.94,2960.92,3903.63,8332.29
2005-10,17035.0,5124.484780,1196.230817,153.94,4656.63,4656.63,5944.68,9160.88
2010-15,16242.0,6697.566150,1498.387665,226.98,5345.62,6361.73,7853.98,11309.73
2015-20,19144.0,10068.989970,1930.314750,339.79,8992.02,9503.32,10751.32,18626.50


In [76]:
wind['t_rd_ft'] = wind['t_rd']*3.2808

In [77]:
wind_old_geo['rotor_dia_ft'] = wind_old_geo['rotor_dia']*3.2808

In [78]:
wind_2014_rotor_d = wind_old_geo.groupby('year_group').rotor_dia_ft.describe()

In [79]:
wind_rotor_d = wind.groupby('year_group').t_rd_ft.describe()

In [80]:
wind_2014_rotor_d

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,4179.0,55.119246,6.010992,42.65040,55.7736,55.77360,55.7736,108.2664
1985-90,1383.0,60.171722,14.732731,52.49280,52.4928,55.77360,55.7736,108.2664
1990-95,416.0,91.996471,7.585188,75.45840,88.5816,88.58160,88.5816,108.2664
1995-00,1306.0,151.068531,22.253667,49.21200,154.1976,157.47840,164.0400,216.5328
2000-05,3801.0,194.200746,38.765935,49.21200,154.1976,201.44112,231.2964,295.2720
2005-10,17046.0,261.310581,30.919727,45.93120,252.6216,252.62160,270.6660,331.3608
2010-15,13510.0,297.284507,35.173344,12.13896,270.6660,295.27200,328.0800,383.8536


In [81]:
wind_rotor_d

,count,mean,std,min,25%,50%,75%,max
year_group,,,,,,,,
1980-85,586.0,51.096500,1.654839,50.19624,50.19624,50.19624,50.19624,54.1332
1985-90,509.0,59.402461,11.593050,36.08880,52.49280,55.77360,63.64752,154.1976
1990-95,339.0,90.033281,7.843018,88.58160,88.58160,88.58160,88.58160,137.7936
1995-00,1170.0,154.714396,19.399780,43.96272,154.19760,157.47840,164.04000,216.5328
2000-05,3879.0,197.539006,46.166823,49.21200,154.19760,201.44112,231.29640,337.9224
2005-10,17035.0,262.914530,33.249449,45.93120,252.62160,252.62160,285.42960,354.3264
2010-15,16242.0,300.828797,35.920109,55.77360,270.66600,295.27200,328.08000,393.6960
2015-20,19144.0,369.700077,36.257006,68.24064,351.04560,360.88800,383.85360,505.2432


In [82]:
wind_2014_rotor_d = wind_2014_rotor_d.iloc[0:5].reset_index()

In [83]:
wind_new_rotor_d = wind_rotor_d.iloc[5:].reset_index()

In [84]:
rotor_d_combo = wind_2014_rotor_d.append(wind_new_rotor_d)

In [85]:
rotor_d_combo

,year_group,count,mean,std,min,25%,50%,75%,max
0,1980-85,4179.0,55.119246,6.010992,42.65040,55.7736,55.77360,55.7736,108.2664
1,1985-90,1383.0,60.171722,14.732731,52.49280,52.4928,55.77360,55.7736,108.2664
2,1990-95,416.0,91.996471,7.585188,75.45840,88.5816,88.58160,88.5816,108.2664
3,1995-00,1306.0,151.068531,22.253667,49.21200,154.1976,157.47840,164.0400,216.5328
4,2000-05,3801.0,194.200746,38.765935,49.21200,154.1976,201.44112,231.2964,295.2720
0,2005-10,17035.0,262.914530,33.249449,45.93120,252.6216,252.62160,285.4296,354.3264
1,2010-15,16242.0,300.828797,35.920109,55.77360,270.6660,295.27200,328.0800,393.6960
2,2015-20,19144.0,369.700077,36.257006,68.24064,351.0456,360.88800,383.8536,505.2432


In [86]:
rotor_d_combo.to_csv('rotor_d_combo.csv')